In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Apr  7 22:08:46 2019

@author: Deechean
"""
from cifar10 import cifar10
import os
import tensorflow as tf
from lenet5 import LetNet5
import tf_general as tfg
import numpy as np


FLAGS = tf.flags.FLAGS
try:
    tf.flags.DEFINE_string('f', '', 'kernel')
    tf.flags.DEFINE_integer('epoch', 10, 'epoch')
    tf.flags.DEFINE_integer('batch_size', 128, 'batch size')
    tf.flags.DEFINE_integer('test_size', 10000, 'test size')
    tf.flags.DEFINE_float('lr', 0.001, 'learning rate')
    tf.flags.DEFINE_boolean('restore', False, 'restore from checkpoint and run test')
    print('parameters were defined.')
except:
    print('parameters have been defined.')

parameters were defined.


In [2]:
data = cifar10();
train_image, train_label = data.get_train_batch(FLAGS.batch_size)
test_image, test_label = data.get_test_batch(FLAGS.test_size)



train set length: 40000
test set length: 10000


In [3]:
ckpt_dir = 'ckpt/'
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

In [4]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, 784], name='x_input')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    y_ = tf.cast(train_label, tf.float32)

In [5]:
with tf.name_scope('prediction'):
    le_net5 = LetNet5(x, keep_prob)
    fc2 = le_net5.fc2
    logits = le_net5.logits
    y = le_net5.prediction

Instructions for updating:
dim is deprecated, use axis instead


In [6]:
with tf.name_scope('cross_entropy'):
    logy = tf.log(y)
    y_logy = y*logy
    reduce_sum = tf.reduce_sum(y_logy)
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y)))

In [7]:
with tf.name_scope('train_step'):
    train_step = tf.train.AdagradOptimizer(FLAGS.lr).minimize(cross_entropy)

In [8]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [11]:
if __name__ == '__main__':
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        saver = tf.train.Saver(max_to_keep = 1)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess, coord)
        for i in range(FLAGS.epoch):
            train_image_batch, train_label_batch = cifar10.get_train_batch(FLAGS.batch_size)
            loss, _ , accuracy_rate = sess.run([cross_entropy,train_step,accuracy], 
                                                   feed_dict={keep_prob: 0.5, x:train_image_batch}) 
            
            str_pre=''
            log = []
            log.append(str_pre)
            print('iter:' + str(i), 'accuracy='+str(accuracy_rate))
            
            if i % 10 == 0:  #保存预测模型
                saver.save(sess,'ckpt/cifar10_'+str(i)+'.ckpt',global_step=i)  
                print('iter:' + str(i), str(round(accuracy_rate*100,2))+'%')           
        tf.reset_default_graph()

AttributeError: 'int' object has no attribute 'train_images'

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled
